<a href="https://colab.research.google.com/github/rociolopeth/deccan-traps-gee/blob/main/deccan_traps_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q geemap


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.3 MB/s eta 0:00:00


In [2]:
import ee
import geemap


In [3]:
import ee
ee.Authenticate()

In [4]:
ee.Initialize(project='lips-482716')
print(ee.String('Hello from the Earth Engine servers!').getInfo())

Hello from the Earth Engine servers!


This code block initializes a `geemap` map object, which will be used to visualize geospatial data. It also defines an approximate Region of Interest (ROI) for the Deccan Traps using latitude and longitude coordinates. This ROI will be used to filter and center satellite imagery on the study area.

In [5]:
map = geemap.Map()
# Approximate bounding box of the Deccan Traps
deccan_roi = ee.Geometry.Rectangle([
    60.0, 10.0,   # lon_min, lat_min (Expanded)
    90.0, 30.0    # lon_max, lat_max (Expanded)
])


This code block processes a KML/KMZ file, specifically `Deccan_traps_georreferenced.kmz`. It first converts the KML data into a GeoJSON format. Then, it preprocesses this GeoJSON to remove any altitude (third) coordinate values, ensuring compatibility with Earth Engine, which typically expects 2D geometries. Finally, the preprocessed GeoJSON is converted into an Earth Engine FeatureCollection and added as a layer to the map with a purple outline.

In [10]:
import os
import json

deccan_traps_kml_path = '/content/Deccan_traps_georreferenced.kmz'
output_geojson_path = '/content/deccan_traps_temp.geojson'
try:
    # Convert KML/KMZ to GeoJSON
    geemap.kml_to_geojson(deccan_traps_kml_path, output_geojson_path)

    # Read the GeoJSON content
    with open(output_geojson_path, 'r') as f:
        geojson_content = json.load(f)

    # Preprocess GeoJSON to remove the third coordinate (altitude)
    # This iterates through all features and their geometries to remove the altitude.
    if geojson_content and 'features' in geojson_content:
        for feature in geojson_content['features']:
            if 'geometry' in feature and feature['geometry'] and 'coordinates' in feature['geometry']:
                geom_type = feature['geometry']['type']
                if geom_type == 'Polygon':
                    # For a Polygon, coordinates are [exterior_ring, interior_ring1, ...]
                    # Each ring is a list of [lon, lat, alt] coordinates
                    new_coordinates = []
                    for ring in feature['geometry']['coordinates']:
                        new_ring = [[coord[0], coord[1]] for coord in ring]
                        new_coordinates.append(new_ring)
                    feature['geometry']['coordinates'] = new_coordinates
                elif geom_type == 'MultiPolygon':
                    new_multipolygon_coordinates = []
                    for polygon in feature['geometry']['coordinates']:
                        new_polygon_rings = []
                        for ring in polygon:
                            new_ring = [[coord[0], coord[1]] for coord in ring]
                            new_polygon_rings.append(new_ring)
                        new_multipolygon_coordinates.append(new_polygon_rings)
                    feature['geometry']['coordinates'] = new_multipolygon_coordinates
                elif geom_type == 'LineString':
                    new_coordinates = [[coord[0], coord[1]] for coord in feature['geometry']['coordinates']]
                    feature['geometry']['coordinates'] = new_coordinates
                elif geom_type == 'MultiLineString':
                    new_multiline_coordinates = []
                    for line in feature['geometry']['coordinates']:
                        new_line = [[coord[0], coord[1]] for coord in line]
                        new_multiline_coordinates.append(new_line)
                    feature['geometry']['coordinates'] = new_multiline_coordinates
                elif geom_type == 'Point':
                    feature['geometry']['coordinates'] = [feature['geometry']['coordinates'][0], feature['geometry']['coordinates'][1]]
                elif geom_type == 'MultiPoint':
                    new_multipoint_coordinates = [[coord[0], coord[1]] for coord in feature['geometry']['coordinates']]
                    feature['geometry']['coordinates'] = new_multipoint_coordinates

    # Save the modified GeoJSON back to a temporary file
    with open(output_geojson_path, 'w') as f:
        json.dump(geojson_content, f)
    print(f"Successfully preprocessed and saved GeoJSON: {output_geojson_path}")

    # Now attempt to convert the modified GeoJSON to an Earth Engine object
    deccan_traps_feature = geemap.geojson_to_ee(output_geojson_path)

    # Define a visualization for the KML layer (e.g., a simple purple outline)
    vis_params_kml = {
        'color': '800080',  # Red color
        'pointSize': 5,
        'pointShape': 'circle',
        'width': 2           # Line width
    }

    map.addLayer(deccan_traps_feature, vis_params_kml, 'Deccan Traps KML')
    map

except Exception as e:
    print(f"An error occurred during KML to EE conversion: {e}")
    print("Please ensure your KMZ/KML file contains valid and well-formed geographical features.")
finally:
    # Clean up the temporary GeoJSON file
    if os.path.exists(output_geojson_path):
        os.remove(output_geojson_path)

map.centerObject(deccan_roi, 6)
map

Successfully preprocessed and saved GeoJSON: /content/deccan_traps_temp.geojson


Map(bottom=3871.0, center=[20.392759474197128, 75], controls=(WidgetControl(options=['position', 'transparent_…

This code block is responsible for loading and preparing the COPERNICUS/S2_SR_HARMONIZED collection. It filters the images by date (for the year 2023) and by the defined Region of Interest (Deccan Traps). Finally, it creates a `median composite` from the filtered collection. The purpose of using a median composite is to reduce the impact of cloud cover and other atmospheric effects, resulting in a clearer, more representative image of the study area over the specified time period.

In [11]:
image = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
    .filterDate('2023-01-01', '2023-12-31')\
    .filterBounds(deccan_roi)\
    .median()


This code block is used to visualize the `median composite` (named `image` in the previous step) in natural color. It defines `vis_params_s2` to map the Sentinel-2 bands B4 (Red), B3 (Green), and B2 (Blue) to the corresponding display channels, providing a true-color representation of the Earth's surface. A new `geemap` map object, `map_natural`, is created and centered on the `deccan_roi`. Finally, the natural color composite is added as a layer to this map for visualization.

In [12]:


# Define visualization parameters for natural color (B4, B3, B2)
vis_params_s2 = {
    'min': 0,
    'max': 2000,
    'bands': ['B4', 'B3', 'B2']
}
map_natural = geemap.Map()
map_natural.centerObject(deccan_roi, 6)

map_natural.addLayer(image, vis_params_s2, 'Sentinel-2 Natural Color Composite')
map_natural

Map(center=[20.392759474197128, 75], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Se…

This code block is used to visualize a false-color composite image by mapping the Shortwave Infrared (SWIR, B12), Near-Infrared (NIR, B8), and Red (B4) bands of Sentinel-2 to the RGB display channels. This specific band combination (SWIR-NIR-Red) is particularly useful for differentiating various land cover features, such as vegetation, bare soil, and water bodies. It defines `vis_params_swir_nir_red` with a specified minimum and maximum value range (0 to 3000) for appropriate scaling. A new `geemap` map object, `map_swir_nir_red`, is then created, centered on the `deccan_roi` (Deccan Traps Region of Interest), and the false-color composite image is added as a layer for visualization.

In [13]:
# Define visualization parameters for false color SWIR-NIR-Red (B12, B8, B4)
vis_params_swir_nir_red = {
    'min': 0,
    'max': 3000,
    'bands': ['B12', 'B8', 'B4']
}
map_swir_nir_red = geemap.Map()
map_swir_nir_red.centerObject(deccan_roi, 6)

map_swir_nir_red.addLayer(image, vis_params_swir_nir_red, 'Sentinel-2 SWIR-NIR-Red Composite')
map_swir_nir_red

Map(center=[20.392759474197128, 75], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Se…

In [14]:
# Define visualization parameters for false color NIR-Red-Green (B8, B4, B3)
vis_params_nir_red_green = {
    'min': 0,
    'max': 3000,
    'bands': ['B8', 'B4', 'B3']
}
map_nir_red_green = geemap.Map()
map_nir_red_green.centerObject(deccan_roi, 6)
map_nir_red_green.addLayer(image, vis_params_nir_red_green, 'Sentinel-2 NIR-Red-Green Composite')
map_nir_red_green

Map(center=[20.392759474197128, 75], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Se…

In [16]:
## 🧹 Notebook maintenance (run before pushing to GitHub)

import json

# notebook path
notebook_path = "/content/drive/MyDrive/Colab Notebooks/deccan_traps_visualization.ipynb"

try:
    # Open notebook
    with open(notebook_path, "r", encoding="utf-8") as f:
        nb = json.load(f)

    # delete metadata.widgets
    if "widgets" in nb.get("metadata", {}):
        del nb["metadata"]["widgets"]
        print("✔ metadata.widgets deleted")
    else:
        print("✔ widgets not found in metadata")

except Exception as e:
    print("❌ Error cleaning metadata.widgets:", e)


✔ metadata.widgets eliminado correctamente
